SELF_ASK_WITH_REACT_AGENT USING TOOLS

In [1]:
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain import PromptTemplate

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [4]:
embedding=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\user\OneDrive\Desktop\langgraph\myvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [6]:
llm=ChatGroq(model="gemma2-9b-it")

In [7]:
llm.invoke("what is gen ai")

AIMessage(content="##  Gen AI: Unleashing the Power of Generative Artificial Intelligence\n\nGen AI, short for Generative Artificial Intelligence, is a fascinating branch of AI that focuses on creating new content rather than simply analyzing existing data. Imagine a computer program that can compose music, write stories, design images, or even generate code – that's the power of Gen AI.\n\n**Here's a breakdown:**\n\n* **How it Works:** Gen AI models are typically trained on massive datasets of text, code, images, audio, or other types of data. Through complex algorithms, they learn the underlying patterns and structures within this data. This learned knowledge allows them to generate new content that resembles the training data but is original and unique.\n* **Key Techniques:**\n\n    * **Generative Adversarial Networks (GANs):** Two neural networks, a generator and a discriminator, compete against each other. The generator tries to create realistic content, while the discriminator tr

In [8]:
loader=PyPDFLoader(file_path="C:\\Users\\user\\OneDrive\\Desktop\\langgraph\\temp.pdf")
docs=loader.load()

In [9]:
documents=RecursiveCharacterTextSplitter(chunk_size=400,chunk_overlap=40).split_documents(docs)

In [10]:
documents

[Document(metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\langgraph\\temp.pdf', 'page': 0}, page_content='From Wikipedia, the free encyclopedia \nAn illustration of main components of the \ntransformer model from the paper \n"Attention Is All You Need"[1] is a 2017 landmark[2][3] research paper in machine learning authored by \neight scientists working at Google. The paper introduced a new deep learning architecture known as'),
 Document(metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\langgraph\\temp.pdf', 'page': 0}, page_content='the transformer, based on the attention mechanism proposed in 2014 by Bahdanau et al.[4] It is \nconsidered a foundational[5] paper in modern artificial intelligence, as the transformer approach has \nbecome the main architecture of large language models like those based on GPT.[6][7] At the time, the'),
 Document(metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\langgraph\\temp.pdf', 'page': 0}, page_content='focus of the research w

In [11]:
db=FAISS.from_documents(documents=documents,embedding=embedding)

In [12]:
vector_db=db.as_retriever()

In [13]:
input="what is transformer"

In [14]:
vector_db.get_relevant_documents(input)

C:\Users\user\AppData\Local\Temp\ipykernel_15276\1757252538.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  vector_db.get_relevant_documents(input)


[Document(id='2e9b0301-2ebe-4bdd-be16-63143e90d026', metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\langgraph\\temp.pdf', 'page': 0}, page_content='"Transformer" was picked because Uszkoreit liked the sound of that word.[9] \nAn early design document was titled "Transformers: Iterative Self-Attention and Processing for \nVarious Tasks", and included an illustration of six characters from the Transformers animated show. \nThe team was named Team Transformer.[8]'),
 Document(id='ee7268c4-8083-4207-b590-838a8a5ea0f4', metadata={'source': 'C:\\Users\\user\\OneDrive\\Desktop\\langgraph\\temp.pdf', 'page': 0}, page_content='Some early examples that the team tried their Transformer architecture on included English-to-\nGerman translation, generating Wikipedia articles on "The Transformer", and parsing. These \nconvinced the team that the Transformer is a general purpose language model, and not just good for \ntranslation.[9] \nAs of 2024, the paper has been cited more than 100,000 t

In [15]:
from langchain.tools.retriever import create_retriever_tool

In [16]:
rag_tool=create_retriever_tool(
                                vector_db,
                                "Intermediate Answer",
                                "it give some information about the transformer architecture"
                                )

In [17]:
rag_tool.name

'Intermediate Answer'

In [18]:
tools=[rag_tool]

In [19]:
from langchain.agents import AgentExecutor,create_self_ask_with_search_agent

In [20]:
from langchain import hub

In [21]:
prompt=hub.pull("hwchase17/self-ask-with-search")

In [22]:
print(prompt.template,end=" ")

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [23]:
agent=create_self_ask_with_search_agent(llm=llm,tools=tools,prompt=prompt)

In [24]:
chain=AgentExecutor(agent=agent,tools=tools,return_intermediate_steps=True,verbose=True,handle_parsing_errors=True)

In [25]:
response1=chain.invoke({"input":"what is transformer in  Artificial Intelligence"})



> Entering new AgentExecutor chain...
Could not parse output: Yes, follow-up questions are needed.  

Here's why and some examples:

* **"Transformer" is a broad term:** In AI, "transformer" can refer to the Transformer architecture, a specific type of neural network, or even broader concepts related to language modeling.  

**Follow-up questions could clarify:**

* **Specific focus:** "Are you asking about the Transformer architecture used in language models?" 
* **Level of detail:** "Do you want a basic explanation of what transformers are, or are you interested in technical details about how they work?"
* **Application:** "Are you interested in transformers for text generation, machine translation, or another application?"


Let me know if you'd like to explore a specific aspect of transformers! 


For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE Invalid or incomplete responseCould not parse output: You're right! My respon